In [2]:
import os
import json
import datetime

#use whichever folder
folder = #use whichever folder the files are stored in

#quick function to open files
def open_file(file_name):
    with open(os.path.join(folder, file_name), mode = 'r', encoding = 'utf8') as reader:
        dct = json.load(reader)
    return dct

In [4]:
#Open Weekend Lineup
weekend_lineup_dict = open_file('Weekend Artist Lineup.json')
weekend_lineup_dict

{'Childish Gambino': {'Day': 'Friday, April 19, 2019',
  'Start Time': 'Friday, April 19, 2019 11:25 PM -0700',
  'End Time': 'Saturday, April 20, 2019 12:55 AM -0700',
  'Stage': ' Coachella Stage'},
 'Yves Tumor': {'Day': 'Friday, April 19, 2019',
  'Start Time': 'Saturday, April 20, 2019 12:00 AM -0700',
  'End Time': 'Saturday, April 20, 2019 01:30 AM -0700',
  'Stage': ' Gobi'},
 'Kayzo': {'Day': 'Friday, April 19, 2019',
  'Start Time': 'Saturday, April 20, 2019 12:05 AM -0700',
  'End Time': 'Saturday, April 20, 2019 01:35 AM -0700',
  'Stage': ' Sahara'},
 'Hot Since 82/Lauren Lane/Nic Fanciulli': {'Day': 'Friday, April 19, 2019',
  'Start Time': 'Friday, April 19, 2019 11:00 PM -0700',
  'End Time': 'Saturday, April 20, 2019 12:30 AM -0700',
  'Stage': ' Yuma'},
 'DJ Snake': {'Day': 'Friday, April 19, 2019',
  'Start Time': 'Friday, April 19, 2019 10:35 PM -0700',
  'End Time': 'Saturday, April 20, 2019 12:05 AM -0700',
  'Stage': ' Outdoor Theatre'},
 'Nora En Pure': {'Day': 

In [8]:
#Get set info for artist of interest
def get_artist_set_info_dict(artist_name):
    return open_file('Weekend Artist Lineup.json')[artist_name]

get_artist_set_info_dict('DJ Snake')

{'Day': 'Friday, April 19, 2019',
 'Start Time': 'Friday, April 19, 2019 10:35 PM -0700',
 'End Time': 'Saturday, April 20, 2019 12:05 AM -0700',
 'Stage': ' Outdoor Theatre'}

In [7]:
#tweet timestamp to time converter
def get_tweet_timestamp(tid):
    offset = 1288834974657
    tstamp = (tid >> 22) + offset
    utcdttime = datetime.datetime.fromtimestamp(tstamp/1000, tz=datetime.timezone.utc)
    return utcdttime

In [37]:
#Get list of tweets from artist set
def get_tweets_from_set(artist_name):
    artist_set_info_dict = get_artist_set_info_dict(artist_name)
    hour_tweet_list = []
    artist_tweet_list = []
    
    start_time = datetime.datetime.strptime(artist_set_info_dict['Start Time'], '%A, %B %d, %Y %I:%M %p %z')
    start_day = start_time.utctimetuple().tm_mday
    start_hour = start_time.utctimetuple().tm_hour
    
    
    end_time = datetime.datetime.strptime(artist_set_info_dict['End Time'], '%A, %B %d, %Y %I:%M %p %z')
    end_day = end_time.utctimetuple().tm_mday
    end_hour = end_time.utctimetuple().tm_hour
    
    first_hour = f'{start_day}.{start_hour}'
    last_hour = f'{end_day}.{end_hour}'
    
    if start_day == end_day:
        day_dct = open_file('4.' + str(start_day) + '.19 coachella tweets.json')
        if end_hour - start_hour == 0:
            hour_tweet_list = [day_dct[first_hour]]
        elif end_hour - start_hour == 1:
            hour_tweet_list = [day_dct[first_hour], day_dct[last_hour]]
        else:
            hour_tweet_list = [day_dct[first_hour], day_dct[f'{start_day}.{start_hour + 1}'], day_dct[last_hour]]
    else:
        day_start_dct = open_file('4.' + str(start_day) + '.19 coachella tweets.json')
        day_end_dct = open_file('4.' + str(end_day) + '.19 coachella tweets.json')
        if end_hour - start_hour == -23:
            hour_tweet_list = [day_start_dct[first_hour], day_end_dct[last_hour]] # for when tweet hour ranges from hour 23 to hour 0
        else:
            if start_hour == 23:
                hour_tweet_list = [day_start_dct[first_hour], day_end_dct[f'{end_day}.{end_hour-1}'], day_end_dct[last_hour]] #for when tweet hour ranges from hour 23 - hour 1
            else:
                hour_tweet_list = [day_start_dct[first_hour], day_start_dct[f'{start_day}.{start_hour+1}'], day_end_dct[last_hour]] #for when tweet hour ranges from hour 22 - hour 0
        
    hour_tweet_list = [element for row in hour_tweet_list for element in row]
    hour_tweet_list = sorted(hour_tweet_list)
    
    for tweet_id in hour_tweet_list:
        if start_time < get_tweet_timestamp(int(tweet_id)) < end_time:
            artist_tweet_list.append(tweet_id)
    
        
    return artist_tweet_list
      

In [39]:
dj_snake = get_tweets_from_set('DJ Snake')

In [40]:
get_tweet_timestamp(int(dj_snake[0])), get_tweet_timestamp(int(dj_snake[-1]))

(datetime.datetime(2019, 4, 20, 5, 35, 0, 215000, tzinfo=datetime.timezone.utc),
 datetime.datetime(2019, 4, 20, 7, 4, 59, 264000, tzinfo=datetime.timezone.utc))

In [41]:
len(dj_snake)

35849

Now that you have the list of tweet_ids you can pull the full tweet objects using Twython

In [42]:
from twython import Twython
from time import sleep
import itertools

#consumer_key = add consumer key here
#consumer_secret = add consumer secret here
#access_token = add access token here
#access_token_secret = add access token secret here

#twitter = Twython(consumer_key, consumer_secret, access_token, access_token_secret)

In [31]:
status = twitter.lookup_status(id = dj_snake[0], tweet_mode='extended')
status

[{'created_at': 'Sat Apr 20 05:35:00 +0000 2019',
  'id': 1119474524863574016,
  'id_str': '1119474524863574016',
  'full_text': '#KeanuReeves Kindness: Actor Entertains &amp; Transports Passengers home, after Their Plane Makes Emergency Landing https://t.co/Mq14UwF4Gr',
  'truncated': False,
  'display_text_range': [0, 139],
  'entities': {'hashtags': [{'text': 'KeanuReeves', 'indices': [0, 12]}],
   'symbols': [],
   'user_mentions': [],
   'urls': [{'url': 'https://t.co/Mq14UwF4Gr',
     'expanded_url': 'https://buff.ly/2PmSDlI',
     'display_url': 'buff.ly/2PmSDlI',
     'indices': [116, 139]}]},
  'source': '<a href="https://buffer.com" rel="nofollow">Buffer</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 125481375,
   'id_str': '125481375',
   'name': '2getaticket.com',
   'screen_name': '2getaticket',
   'location': 'London UK',
   'd

In [32]:
#twitter allows you to request up to 100 unique tweet ids in a lookup_status request.
#function to group by 100 

def grouper(iterable, n=100):
    it = iter(iterable)
    while True:
        chunk = list(itertools.islice(it, n))
        if not chunk:
            return
        yield chunk

In [33]:
#creating the tweet dictionary.  Using a 1 second sleep timer to not max the rate limit 
tweets_dict = {}
for tweets in grouper(dj_snake, 100):
    rslts = twitter.lookup_status(id=','.join(tweets), tweet_mode='extended')
    for tweet in rslts:
        tweets_dict[tweet['id']] = tweet
    sleep(1)

In [35]:
#tweets_dict.keys()

In [36]:
#example tweet
tweets_dict[1119474533105307648]

{'created_at': 'Sat Apr 20 05:35:02 +0000 2019',
 'id': 1119474533105307648,
 'id_str': '1119474533105307648',
 'full_text': 'rosé take the jacket off!!!!!!',
 'truncated': False,
 'display_text_range': [0, 30],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1161253273,
  'id_str': '1161253273',
  'name': 'el',
  'screen_name': 'rosesarerosie',
  'location': 'england ',
  'description': 'twicepink nation! ~ Harry',
  'url': 'https://t.co/ZiCeNxAT7p',
  'entities': {'url': {'urls': [{'url': 'https://t.co/ZiCeNxAT7p',
      'expanded_url': 'https://curiouscat.me/littlegIow',
      'display_url': 'curiouscat.me/littlegIow',
      'indices': [0, 23]}]},
   'description': {'urls': []}}